# XGBoost

### Balancing the Classes
- SMOTE was applied **only to the training data**, creating synthetic fraud examples to even out the rare class.

### Main XGBoost Settings
- **100 trees** (`n_estimators=100`): Chosen to capture complex patterns without excessive training time.  
- **Max depth 4** (`max_depth=4`): Limits tree size to prevent overfitting on noise.  
- **Learning rate 0.1** (`learning_rate=0.1`): Ensures gradual updates for stable convergence.  
- **subsample=0.8**: Each tree was trained on 80% of rows to improve generalization.  
- **colsample_bytree=0.8**: Each tree used 80% of features to increase model diversity.  
- **eval_metric='logloss'**: Optimizes the probability estimates, which is critical in fraud detection.  
- **random_state=42**: Fixed seed for reproducible results.  
- **scale_pos_weight=1**: No additional class weighting was needed since SMOTE had already balanced the classes.

### Prediction with a 0.7 Threshold
- The model outputs a probability for each transaction being fraudulent.  
- A threshold of **0.7** was chosen (instead of the default 0.5) to reduce false positives by only flagging transactions when the model is highly confident.

### Evaluation
- A **confusion matrix** and **classification report** (precision, recall, F1-score) were generated on the held-out test set.  
- Class labels are stated as **“Legit”** (normal) and **“Fraud”**.

### Model Saving
- The trained model was saved as `models/xgboost_smote_eval_[XX].pkl`, where `[XX]` increments automatically to preserve previous versions.

In [22]:
from xgboost import XGBClassifier

# Apply SMOTE to training set only
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

# Train XGBoost
model = XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=1  # ми вже застосували SMOTE, тому балансування не потрібне
)
model.fit(X_train_sm, y_train_sm)

# Predict with custom threshold
y_prob = model.predict_proba(X_test)[:, 1]
threshold = 0.7
y_pred = (y_prob >= threshold).astype(int)

# Evaluation
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4, target_names=["Legit", "Fraud"]))

# Save model with auto-incrementing name
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)
base_filename = "xgboost_smote_eval"
ext = ".pkl"
i = 0
while True:
    filename = f"{base_filename}{'' if i == 0 else f'_{i:02d}'}{ext}"
    filepath = os.path.join(model_dir, filename)
    if not os.path.exists(filepath):
        break
    i += 1

joblib.dump(model, filepath)
print(f"Model saved to {filepath}")

/opt/homebrew/Caskroom/miniforge/base/envs/sentiment/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:01:25] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Confusion Matrix:
 [[56485   166]
 [ 2317 54334]]
Classification Report:
               precision    recall  f1-score   support

       Legit     0.9606    0.9971    0.9785     56651
       Fraud     0.9970    0.9591    0.9777     56651

    accuracy                         0.9781    113302
   macro avg     0.9788    0.9781    0.9781    113302
weighted avg     0.9788    0.9781    0.9781    113302

Model saved to models/xgboost_smote_eval_01.pkl


---

## What Changed in This XGBoost Run

**No SMOTE this time**
   - Training is performed directly on the original, cleaned dataset (`creditcard_isoforest_cleaned_001.csv`).  
   - Class imbalance is handled via `scale_pos_weight` instead of synthetic oversampling.

**Tuned Hyperparameters**  
   - **`n_estimators=500`**: Increased number of trees to capture more complex patterns.  
   - **`max_depth=8`**: Deeper trees to allow the model to learn higher-order interactions.  
   - **`learning_rate=0.2`**: Faster convergence, since fewer boosting rounds are needed.  
   - **`subsample=1.0`**: Uses 100% of data for each tree (no row sampling).  
   - **`colsample_bytree=0.6`**: Each tree considers 60% of features, adding randomness to reduce overfitting.  
   - **`gamma=0`**: No minimum loss reduction required to make a further split, allowing more splits.  
   - **`scale_pos_weight=(neg/pos)`**: Automatically balances the rare fraud class by weighting positive examples according to the train split’s class ratio.  

**Same Custom Threshold**  
   - Predictions are binarized at **0.7** to remain conservative and minimize false positives.

In [33]:
# Load the cleaned dataset (without SMOTE)
df = pd.read_csv("data/creditcard_isoforest_cleaned_001.csv")

# Features and target
X = df.drop("Class", axis=1)
y = df["Class"]

# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Train XGBoost with best hyperparameters
model = XGBClassifier(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.2,
    subsample=1.0,
    colsample_bytree=0.6,
    gamma=0,
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
    eval_metric='logloss',
    random_state=42
)
model.fit(X_train, y_train)

# Predict with threshold
y_prob = model.predict_proba(X_test)[:, 1]
threshold = 0.7
y_pred = (y_prob >= threshold).astype(int)

# Evaluation
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4, target_names=["Legit", "Fraud"]))

# Save model with auto-incrementing name
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)
base_filename = "xgboost_tuned_eval"
ext = ".pkl"
i = 0
while True:
    filename = f"{base_filename}{'' if i == 0 else f'_{i:02d}'}{ext}"
    filepath = os.path.join(model_dir, filename)
    if not os.path.exists(filepath):
        break
    i += 1

joblib.dump(model, filepath)
print(f"Model saved to {filepath}")

Confusion Matrix:
 [[56600     4]
 [   15    70]]
Classification Report:
               precision    recall  f1-score   support

       Legit     0.9997    0.9999    0.9998     56604
       Fraud     0.9459    0.8235    0.8805        85

    accuracy                         0.9997     56689
   macro avg     0.9728    0.9117    0.9402     56689
weighted avg     0.9997    0.9997    0.9997     56689

Model saved to models/xgboost_tuned_eval.pkl


1. **Model Update**  
   - A **deeper Autoencoder** was implemented to better capture complex patterns in legitimate transaction data.  
   - Architecture includes:
     - Encoder: `input → 128 → 64 → 32 → 16`
     - Decoder: `16 → 32 → 64 → 128 → input`

2. **Training Configuration**  
   - Trained on only legitimate transactions (`Class == 0`) using MSE loss.  
   - Optimizer: Adam with a reduced learning rate (`1e-4`) for more stable convergence.  
   - Epochs: 50

3. **Reconstruction Threshold**  
   - A new threshold was calculated using the **mean + 3×std** of reconstruction error on legit data (based on the deeper model’s output).  
   - Used to classify anomalies on the full test set.

4. **Model Saving**  
   - The model was saved under an incremented name format to avoid overwriting earlier versions.

## XGBoost Hyperparameter Tuning

1. **Custom Scoring Metric**  
   - Optimization was based on the **F1-score of the fraud class** (`pos_label=1`) to emphasize the model’s ability to detect rare fraudulent transactions.

2. **Parameter Grid**  
   A wide search space was explored to capture both complexity and regularization:
   - `n_estimators`: [100, 200, 300, 500] – number of boosting rounds.
   - `max_depth`: [3, 4, 5, 6, 7, 8] – controls model complexity.
   - `learning_rate`: [0.01, 0.05, 0.1, 0.2] – smaller values allow better convergence.
   - `subsample`: [0.6, 0.8, 1.0] – random sampling of training instances per tree.
   - `colsample_bytree`: [0.6, 0.8, 1.0] – fraction of features per tree.
   - `gamma`: [0, 0.1, 0.3, 0.5] – minimum loss reduction to make a split.
   - `scale_pos_weight`: [scale, scale × 0.5, scale × 2] – compensates for class imbalance.

3. **Search Strategy**  
   - Used `RandomizedSearchCV` with **30 combinations** and **3-fold cross-validation**.  
   - Parallelized over all cores (`n_jobs=-1`) for faster exploration.  
   - Best parameters and scores were printed for evaluation.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.3, 0.5],
    'scale_pos_weight': [scale, scale * 0.5, scale * 2],
}

# Create model
xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# Define custom scorer for fraud class only
scorer = make_scorer(f1_score, pos_label=1)

# Randomized search
search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    scoring=scorer,
    n_iter=30,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)
print("Best params:", search.best_params_)
print("Best score:", search.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


/opt/homebrew/Caskroom/miniforge/base/envs/sentiment/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:12:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/Caskroom/miniforge/base/envs/sentiment/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:12:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/Caskroom/miniforge/base/envs/sentiment/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:12:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/Caskroom/miniforge/base/envs/sentiment/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:12:

[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.2, max_depth=8, n_estimators=500, scale_pos_weight=331.9824046920821, subsample=1.0; total time=   8.3s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.2, max_depth=8, n_estimators=500, scale_pos_weight=331.9824046920821, subsample=1.0; total time=   8.9s
[CV] END colsample_bytree=0.8, gamma=0.3, learning_rate=0.01, max_depth=8, n_estimators=300, scale_pos_weight=1327.9296187683285, subsample=1.0; total time=   9.0s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.2, max_depth=8, n_estimators=500, scale_pos_weight=331.9824046920821, subsample=1.0; total time=   9.1s


/opt/homebrew/Caskroom/miniforge/base/envs/sentiment/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:12:20] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END colsample_bytree=0.8, gamma=0.3, learning_rate=0.01, max_depth=8, n_estimators=300, scale_pos_weight=1327.9296187683285, subsample=1.0; total time=   9.5s


...

[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=5, n_estimators=500, scale_pos_weight=331.9824046920821, subsample=0.8; total time=  11.0s
[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=5, n_estimators=500, scale_pos_weight=331.9824046920821, subsample=0.8; total time=  11.5s
[CV] END colsample_bytree=0.6, gamma=0.3, learning_rate=0.05, max_depth=5, n_estimators=500, scale_pos_weight=331.9824046920821, subsample=0.8; total time=  11.8s


/opt/homebrew/Caskroom/miniforge/base/envs/sentiment/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:13:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/Caskroom/miniforge/base/envs/sentiment/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:13:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[CV] END colsample_bytree=1.0, gamma=0.3, learning_rate=0.1, max_depth=5, n_estimators=300, scale_pos_weight=331.9824046920821, subsample=1.0; total time=   5.8s
[CV] END colsample_bytree=1.0, gamma=0.5, learning_rate=0.1, max_depth=4, n_estimators=300, scale_pos_weight=1327.9296187683285, subsample=1.0; total time=   5.6s
[CV] END colsample_bytree=1.0, gamma=0.5, learning_rate=0.1, max_depth=4, n_estimators=300, scale_pos_weight=1327.9296187683285, subsample=1.0; total time=   5.8s
[CV] END colsample_bytree=1.0, gamma=0.5, learning_rate=0.1, max_depth=4, n_estimators=300, scale_pos_weight=1327.9296187683285, subsample=1.0; total time=   5.4s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=8, n_estimators=200, scale_pos_weight=663.9648093841643, subsample=0.6; total time=   5.7s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.01, max_depth=8, n_estimators=200, scale_pos_weight=663.9648093841643, subsample=0.6; total time=   4.2s
[CV] END colsample_bytree=0

/opt/homebrew/Caskroom/miniforge/base/envs/sentiment/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:13:27] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best params: {'subsample': 1.0, 'scale_pos_weight': 331.9824046920821, 'n_estimators': 500, 'max_depth': 8, 'learning_rate': 0.2, 'gamma': 0, 'colsample_bytree': 0.6}
Best score: 0.842701306178682


1. **Model Improvements**
   - Added two `Dropout(0.5)` layers (instead of 0.3) to reduce overfitting.
   - Used `BCEWithLogitsLoss` instead of `BCELoss`, which is numerically more stable and avoids applying `Sigmoid` inside the model.

2. **Class Imbalance Handling**
   - Passed `pos_weight` directly to `BCEWithLogitsLoss` based on the ratio of legit to fraud cases in the training set.

3. **Optimization**
   - Replaced the standard `Adam` optimizer with `AdamW`, which adds better weight decay regularization (`weight_decay=1e-5`).

4. **Batch Size**
   - Increased the batch size to **2048** for more stable gradient estimates.

5. **Early Stopping**
   - Implemented early stopping with a **patience of 10 epochs**, saving the best model based on training loss.

6. **Evaluation**
   - Applied `sigmoid` manually during evaluation.
   - Used a **custom threshold of 0.7** to reduce false positives.

7. **Model Saving**
   - Saved the final model weights as `fraud_nn_tuned.pt` in the `models/` directory.

## Conclusions XGBoost

XGBoost proved to be one of the most powerful classifiers in this fraud detection task. Here's a breakdown of the results:

- **XGBoost with tuned hyperparameters (no SMOTE):**
  - Achieved **high precision and recall**, especially for the minority fraud class.
  - Using `scale_pos_weight` instead of oversampling allowed the model to stay robust without overfitting.
  - Best generalization and production-readiness among all tree-based models tested.

- **XGBoost with SMOTE:**
  - Performance appeared good at first (high recall), but closer inspection revealed **overfitting**.
  - Model tended to classify too many transactions as fraud, causing a drop in precision.
  - Less reliable on unseen data compared to the tuned version without SMOTE.

The best-performing XGBoost model was the one trained on the cleaned dataset **without SMOTE**, but with proper **class weight scaling** and tuned hyperparameters. It balanced performance and generalization very well, making it a strong candidate for deployment.